# **Proyecto Final: SkinRateAI – Predicción de Ratings para Skincare**

---

### **Alumno:** Valentina Andrea Lorefice
### **Profesor:** Julio Paredes
### **Diplomatura en Python para Data Science - Universidad Tecnológica Nacional**

---


## 🌟 **Introducción al Proyecto: SkinRateAI – Predicción de Ratings para Skincare**

🧴 El proyecto **SkinRateAI** busca proporcionar a la industria cosmética una herramienta avanzada que optimice el **nombre** 🏷️ y **precio** 💰 de los productos de *skincare* para maximizar sus **ventas** 📈.  
El sistema analiza la mención de los **ingredientes activos** 🌿 en el nombre del producto y su precio, y predice el **rating** ⭐ y las **ventas anuales** del producto potencial. Además, sugiere **ajustes inteligentes** para optimizar los ingresos, considerando cómo estos factores influyen en la percepción del consumidor 🧠.

📊 Estudios previos, como los de **McKinsey & Co. (2021)**, han demostrado que las **calificaciones de productos** influyen fuertemente en las ventas. Por ejemplo, un aumento de 0.2 estrellas ⭐ (de 4.2 a 4.4) puede incrementar las ventas hasta en un **24%** 🚀. Este dato subraya cómo **pequeñas mejoras** en el rating pueden tener un **gran impacto**.

💵 El **precio** también es clave en la estrategia de ventas. Ciertos precios actúan como **señales psicológicas** para los consumidores 🛍️. Un precio alto puede sugerir **exclusividad o calidad**, mientras que uno bajo puede causar **desconfianza**. Encontrar el **punto óptimo** es parte del análisis que realiza SkinRateAI ⚖️.

🧪 Por otra parte, el **nombre del producto** cobra relevancia al incluir ingredientes populares como “niacinamide” o “retinol” 🧬. Los consumidores informados buscan ingredientes específicos que satisfagan sus necesidades, y un buen nombre puede **mejorar la visibilidad del producto** 🔍 (Euromonitor, 2020).

🤖 Con **SkinRateAI**, las empresas podrán **anticipar la recepción de un producto** antes de su lanzamiento, y definir **nombres y precios óptimos** basados en análisis de datos. El modelo se alimenta de información histórica en plataformas como *Sephora* 🛒, detectando patrones y realizando **recomendaciones precisas**. Así, se convierten datos en decisiones estratégicas, **minimizando riesgos** y **maximizando oportunidades** 🎯.

---

### 📚 **Fuentes:**

- [📄 McKinsey & Co. – Five-star growth](https://www.mckinsey.com/industries/consumer-packaged-goods/our-insights/five-star-growth-using-online-ratings-to-design-better-products)
- [📰 How online reviews influence sales – Spiegel](https://spiegel.medill.northwestern.edu/how-online-reviews-influence-sales/?utm_source=chatgpt.com)
- [📘 Online Reviews Whitepaper – PDF](https://spiegel.medill.northwestern.edu/wp-content/uploads/sites/2/2021/04/Online-Reviews-Whitepaper.pdf?utm_source=chatgpt.com)
- [📈 Online reviews equal revenue – Comalytics](https://comalytics.com/online-reviews-equal-revenue/)
- [🧪 Ingredients before brands – Vogue Business](https://www.voguebusiness.com/beauty/ingredients-before-brands-the-new-beauty-consumer-priority)
- [🌿 The rise of ingredient-led beauty – Euromonitor](https://www.euromonitor.com/article/the-rise-of-ingredient-led-beauty)


#Libraries

In [ ]:
from google.colab import drive # Acceder a Google Drive en Google Colab
import pandas as pd # Manipulación y análisis de datos en estructuras tipo DataFrame
import numpy as np # Operaciones matemáticas y manejo de arrays eficientes
import re # Expresiones regulares para procesar y extraer texto
from collections import defaultdict

# Visualizaciones
import plotly.express as px
from IPython.display import display, HTML
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.figure_factory as ff
import plotly.graph_objects as go
from plotly.subplots import make_subplots

!pip install fuzzywuzzy # Instalación de fuzzywuzzy para coincidencia difusa de texto
from fuzzywuzzy import fuzz # Comparación de similitud entre cadenas de texto
from fuzzywuzzy import process # Utiliza fuzzywuzzy para encontrar las mejores coincidencias entre cadenas de texto
from sklearn.metrics import make_scorer
from sklearn.metrics.pairwise import cosine_similarity
import xgboost as xgb # Implementación algoritmo XGBoost
from sklearn.ensemble import RandomForestRegressor # Modelo Random Forest
from sklearn.model_selection import train_test_split, RandomizedSearchCV # División de datos y búsqueda de hiperparámetros
# Métricas de evaluación de modelos de regresión
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score, explained_variance_score, mean_absolute_percentage_error, median_absolute_error

/usr/local/lib/python3.11/dist-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


# 📊 **Análisis Exploratorio**

La base de datos utilizada en este proyecto se compone exclusivamente de **productos de skincare disponibles en Sephora** 🧴🛍️  
🔗 [Ver dataset en Kaggle](https://www.kaggle.com/datasets/nadyinky/sephora-products-and-skincare-reviews)

**Sephora** es una de las plataformas de belleza y cosmética más grandes y reconocidas a nivel mundial 🌍, lo que garantiza una **amplia variedad de productos de cuidado de la piel**.  
Esto permite capturar **patrones relevantes** en diferentes categorías y rangos de precio 💡💰.

La base de datos contiene **datos reales** de productos de *skincare*, lo que facilita un **análisis preciso y confiable** 📈🔍.  
Además, incluye información detallada sobre:

- ✨ **Ingredientes activos**
- ⭐ **Valoraciones (ratings)**
- 📝 **Reseñas de usuarios**

Estos elementos son **fundamentales** para el desarrollo del modelo predictivo de **ratings y ventas esperadas** 📊📦.



In [ ]:
# Montamos Google Drive para acceder a los archivos almacenados en él, cargamos la base de datos de productos desde un archivo CSV en Drive
drive.mount('/content/drive')
df = pd.read_csv('/content/drive/MyDrive/product_info.csv')

# Filtro la base de datos para quedarme solo con productos de la categoría "Skincare", y selecciono únicamente las columnas relevantes para el análisis
df_skincare_all = df[df['primary_category'] == 'Skincare'][['product_name', 'brand_name', 'loves_count', 'rating', 'reviews', 'ingredients', 'price_usd']].dropna(subset=['rating', 'loves_count', 'ingredients'])

print(df_skincare_all.dtypes)

# Genero gráficos interactivos para analizar la distribución de las variables
columns = ["price_usd", "reviews", "loves_count", "rating"]
titles = ["Distribución de Precios", "Distribución de Reseñas", "Distribución de Loves", "Distribución de Rating"]
colors = ["red", "blue", "green", "purple"]

fig = make_subplots(rows=1, cols=4, subplot_titles=titles)

for i, col in enumerate(columns):
    fig.add_trace(go.Histogram(x=df_skincare_all[col], marker_color=colors[i], nbinsx=30), row=1, col=i+1)

    stats = df_skincare_all[col].describe()
    stats_text = f"""
    Count: {stats['count']:.0f}<br>
    Mean: {stats['mean']:.2f}<br>
    Std Dev: {stats['std']:.2f}<br>
    Min: {stats['min']:.2f}<br>
    25%: {stats['25%']:.2f}<br>
    Median: {stats['50%']:.2f}<br>
    75%: {stats['75%']:.2f}<br>
    Max: {stats['max']:.2f}
    """

    fig.add_annotation(
        text=stats_text,
        xref="paper",
        yref="paper",
        x=(i + 0.4) / 4,
        y=0.7,
        showarrow=False,
        align="center",
        font=dict(size=12),
        bgcolor="white"
    )

fig.update_layout(
    title_text="Analisis Exploratorio - Sephora Database",
    height=500,
    width=1400,
    showlegend=False
)

fig.show()

print(df_skincare_all[columns].describe())

Mounted at /content/drive
product_name     object
brand_name       object
loves_count       int64
rating          float64
reviews         float64
ingredients      object
price_usd       float64
dtype: object


         price_usd       reviews   loves_count       rating
count  2224.000000   2224.000000  2.224000e+03  2224.000000
mean     58.396254    480.473022  2.855374e+04     4.232174
std      56.236519    905.946485  5.856679e+04     0.466138
min       3.000000      1.000000  0.000000e+00     1.000000
25%      28.000000     37.750000  4.453500e+03     4.020900
50%      44.000000    173.000000  1.053100e+04     4.311450
75%      69.000000    505.250000  2.806750e+04     4.538650
max     449.000000  16118.000000  1.081315e+06     5.000000


Los datos presentan una gran **variabilidad** en todas las variables 📊:

- 💵 **Precios (`price_usd`)**: Oscilan entre `$3` y `$449`, con una **media** de `$58,40` y una **desviación estándar** de `56,24`.  
  🔍 Esto indica una **distribución dispersa** con presencia de **valores extremos**.

- 📝 **Reseñas (`reviews`)**: Promedio de **480**, con un **mínimo de 1** y un **máximo de 16.118**.  
  La desviación estándar de 905 sugiere una **distribución sesgada**, con algunos productos **mucho más populares** que otros.

- ❤️ **Loves (`loves_count`)**: Varía entre **0 y 1.081.315**, con una **media** de `28.553` y una **desviación estándar** de `58.566`.  
  📉 Se trata de una distribución **altamente asimétrica**, con presencia de **valores atípicos significativos**.

- ⭐ **Calificación (`rating`)**: Valores entre **1 y 5**, con una **media de 4,23** y una desviación estándar de `0,46`.  
  Indica **menor dispersión** y una **tendencia hacia calificaciones altas**.

El atributo `loves_count` debe ser **escalado** 🔧 debido a su rango mucho mayor en comparación con otras variables como `rating`.  Si no se ajusta, dominaría el análisis y afectaría negativamente el rendimiento de cualquier modelo predictivo.

Es esperable que la mayoría de los ratings estén por encima de las ⭐ **4 estrellas**, debido a diversos **sesgos cognitivos**:

- 😊 **Sesgo de positividad** y **disonancia cognitiva**: los usuarios justifican sus decisiones con valoraciones altas.
- 👥 **Sesgo de selección**: quienes dejan reseñas suelen ser **clientes satisfechos**.
- 🎯 **Efecto anclaje**: en plataformas con calificaciones promedio altas, los usuarios tienden a ajustar su puntuación en función de lo que ven.



###Estructura
Para estructurar la información y facilitar el modelado 📐, se crearon **dos diccionarios**:

1. 📚 **Ingredientes activos**: contienen ingredientes populares y en tendencia.
2. 💬 **Buzzwords de marketing**: palabras clave utilizadas en descripciones de productos.

Estos diccionarios permiten:

- 📎 Buscar estos términos en los nombres de los productos.
- 🔢 Generar **variables binarias** que indican su presencia (1) o ausencia (0).

Entre los ingredientes más comunes y con alta presencia en redes sociales 📱 y marketing, se encuentran:

- 🧴 Niacinamida  
- 🌙 Retinol  
- 🔗 Péptidos  
- 🍊 Ácido ascórbico (Vitamina C)  
- 💧 Ácido hialurónico  
- 🍃 Ácido glicólico

📈 Según datos de **Google Trends**, estos ingredientes han mostrado un crecimiento notable en búsquedas, reflejando una **mayor demanda** y conciencia del consumidor.

El lenguaje del skincare gira en torno a palabras que transmiten beneficios deseados ✨.  
Algunas de las más frecuentes son:

- 💧 *hydrate*
- ✨ *glow*
- 🧬 *firm*
- ⏳ *anti-aging*
- 🛠️ *repair*
- 🧼 *detox*
- 🛡️ *protect*

Estas palabras no solo destacan atributos del producto, sino que también influyen en la **percepción del consumidor** 🧠.  
Su análisis permite detectar **tendencias de marketing** y evaluar cómo los productos comunican sus beneficios en función de sus ingredientes.

In [ ]:
# Defino un diccionario de ingredientes clave y sus posibles variaciones en los nombres dentro de la lista de ingredientes
ingredient_dict = {
    'Niacinamide': ['niacinamide', 'vitamin b3', 'b3', 'nicotinamide'],
    'Retinol': ['retinol', 'vitamin a1', 'a1', 'retinyl', 'retinyl acetate', 'retinyl palmitate', 'retinaldehyde', 'hydroxypinacolone retinoate', 'retinoate', 'tretinoin', 'retinoic acid', 'retin-a', 'all-trans retinoic acid'],
    'Peptides': ['peptide', 'peptides', 'oligopeptides', 'polypeptides', 'copper peptides', 'acetyl hexapeptide-8', 'matrixyl'],
    'Ascorbic Acid': ['ascorbic', 'l-ascorbic', 'ascorbic acid', 'vitamin c', 'l-ascorbic acid', 'magnesium ascorbyl phosphate', 'sodium ascorbyl phosphate', 'tetrahexyldecyl ascorbate'],
    'Hyaluronic Acid': ['hyaluronic', 'hyaluronic acid'],
    'Glycolic Acid': ['glycolic', 'glycolic acid', 'alpha hydroxy acid (aha)', 'aha'],
    'Salicylic Acid': ['salicylic', 'salicylic acid', 'bha', 'beta hydroxy acid'],
    'Lactic Acid': ['lactic', 'lactic acid'],
    'Vitamin E': ['vitamin e', 'tocopherol', 'alpha tocopherol'],
    'Ceramides': ['ceramide', 'ceramides', 'lipids', 'phytosphingosine'],
    'Benzoyl Peroxide': ['benzoyl peroxide', 'bp'],
    'Collagen': ['collagen', 'hydrolyzed collagen']
}

# Defino un diccionario de buzzwords clave y sus posibles variaciones en los nombres de productos
buzzword_dict = {
    'Hydrate': ['hydrate', 'hydration', 'moisturize', 'moisturizing', 'water-infused', 'quench', 'dewy', 'plump'],
    'Glow': ['glow', 'radiance', 'illuminate', 'brighten', 'luminous', 'shine', 'light-reflecting', 'even-tone'],
    'Firm': ['firm', 'tighten', 'lifting', 'contouring', 'tone', 'sculpt', 'tightening'],
    'Anti-Aging': ['anti-aging', 'age-defying', 'youthful', 'wrinkle-reducing', 'anti-wrinkle', 'rejuvenate', 'restore', 'age-reversing'],
    'Repair': ['repair', 'restore', 'renew', 'regenerate', 'heal', 'repairing', 'recovery'],
    'Detox': ['detox', 'purify', 'clarify', 'detoxifying', 'cleanse', 'deep-clean', 'refresh'],
    'Protect': ['protect', 'shield', 'defend', 'barrier', 'spf', 'sun protection', 'anti-pollution', 'blue light protection'],
    'Soothing': ['soothing', 'calming', 'relieving', 'cooling', 'comforting', 'anti-inflammatory'],
    'Balance': ['balance', 'oil-control', 'mattifying', 'balancing', 'pore-minimizing', 'sebum control'],
    'Nourish': ['nourish', 'nourishing', 'revitalize', 'replenish', 'feed', 'supply', 'enrich', 'boost'],
    'Brightening': ['brighten', 'even-tone', 'lightening', 'radiant', 'skin-brightening', 'luminosity'],
    'Rejuvenate': ['rejuvenate', 'revitalize', 'refresh', 'renew', 'energize', 'reviving'],
    'Exfoliate': ['exfoliate', 'exfoliating', 'peel', 'scrub', 'polish', 'slough off', 'exfoliation'],
    'Vegan': ['vegan', 'plant-based', 'cruelty-free', 'no animal testing'],
    'Clean': ['clean', 'chemical-free', 'natural', 'organic', 'non-toxic', 'pure', 'green beauty'],
}



Ahora se procede a **buscar los ingredientes activos** en los **nombres de los productos** 🧴 mediante el diccionario previamente creado 🧠.

📌 Se analiza:

- Cuántos productos **mencionan al menos un ingrediente activo** ✅
- Cuántos productos **no mencionan ninguno** ❌
- La **cantidad de productos por cada ingrediente** 🌿

In [ ]:
from collections import defaultdict

# Función para coincidencia difusa
def fuzzy_match(value, terms, threshold=55):
    for term in terms:
        score = process.extractOne(term, [value])[1]  # Calcula la similitud con cada término
        if score >= threshold:
            return term  # Retorna el término si la coincidencia es suficiente
    return None  # Retorna None si no hay coincidencias aceptables

# Función para procesar ingredientes y buzzwords
def process_products(df, ingredient_dict, buzzword_dict):
    # Inicializamos los diccionarios de coincidencias y los contadores
    matches_ingredient_dict = defaultdict(set)
    ingredient_count = defaultdict(int)

    matches_buzzword_dict = defaultdict(set)
    buzzword_count = defaultdict(int)

    # Listas para los productos con y sin ingredientes/buzzwords
    with_ingredients = []
    without_ingredients = []
    with_buzzwords = []
    without_buzzwords = []

    # Iterar sobre los productos y verificar coincidencias
    for idx, row in df.iterrows():
        product_name = str(row['product_name'])

        # Para ingredientes
        matched_ingredients = set()
        for key, terms in ingredient_dict.items():
            matched_term = fuzzy_match(product_name, terms)
            if matched_term:
                matched_ingredients.add(matched_term)
                matches_ingredient_dict[key].add(matched_term)
                ingredient_count[key] += 1  # Contador de productos con este ingrediente

        # Para buzzwords
        matched_buzzwords = set()
        for key, terms in buzzword_dict.items():
            matched_term = fuzzy_match(product_name, terms)
            if matched_term:
                matched_buzzwords.add(matched_term)
                matches_buzzword_dict[key].add(matched_term)
                buzzword_count[key] += 1  # Contador de productos con este buzzword

        # Clasificar productos según si tienen ingredientes o buzzwords
        if matched_ingredients:
            with_ingredients.append(row)
        else:
            without_ingredients.append(row)

        if matched_buzzwords:
            with_buzzwords.append(row)
        else:
            without_buzzwords.append(row)

    # Convertir las listas de productos en DataFrames
    df_with_ingredients = pd.DataFrame(with_ingredients)
    df_without_ingredients = pd.DataFrame(without_ingredients)
    df_with_buzzwords = pd.DataFrame(with_buzzwords)
    df_without_buzzwords = pd.DataFrame(without_buzzwords)

    # Imprimir los términos coincidentes y la cantidad de productos
    print("Términos coincidentes para ingredientes:")
    for ing, matches in matches_ingredient_dict.items():
        print(f"Término(s) para {ing}: {', '.join(matches)}")
        print(f"Cantidad de productos con {ing}: {ingredient_count[ing]}")
        print("-" * 50)

    print("Términos coincidentes para buzzwords:")
    for buzzword, matches in matches_buzzword_dict.items():
        print(f"Término(s) para {buzzword}: {', '.join(matches)}")
        print(f"Cantidad de productos con {buzzword}: {buzzword_count[buzzword]}")
        print("-" * 50)

    return df_with_ingredients, df_without_ingredients, df_with_buzzwords, df_without_buzzwords


# Procesamos los productos
df_with_ingredients, df_without_ingredients, df_with_buzzwords, df_without_buzzwords = process_products(
    df_skincare_all, ingredient_dict, buzzword_dict)

# Cantidad de productos en ambos grupos
size_with_ingredients = df_with_ingredients.shape[0]
size_without_ingredients = df_without_ingredients.shape[0]
size_with_buzzwords = df_with_buzzwords.shape[0]
size_without_buzzwords = df_without_buzzwords.shape[0]

# Imprimir las cantidades de productos en cada grupo
print(f"\nCantidad de productos con ingredientes: {size_with_ingredients}")
print(f"Cantidad de productos sin ingredientes: {size_without_ingredients}")
print(f"Cantidad de productos con buzzwords: {size_with_buzzwords}")
print(f"Cantidad de productos sin buzzwords: {size_without_buzzwords}")


Términos coincidentes para ingredientes:
Término(s) para Collagen: hydrolyzed collagen, collagen
Cantidad de productos con Collagen: 52
--------------------------------------------------
Término(s) para Ceramides: lipids, phytosphingosine, ceramides, ceramide
Cantidad de productos con Ceramides: 389
--------------------------------------------------
Término(s) para Lactic Acid: lactic acid, lactic
Cantidad de productos con Lactic Acid: 312
--------------------------------------------------
Término(s) para Retinol: retinyl palmitate, retinyl, retinoate, retinol, tretinoin, retinaldehyde, vitamin a1, retin-a, retinoic acid, retinyl acetate, all-trans retinoic acid
Cantidad de productos con Retinol: 707
--------------------------------------------------
Término(s) para Niacinamide: niacinamide, vitamin b3, b3
Cantidad de productos con Niacinamide: 274
--------------------------------------------------
Término(s) para Ascorbic Acid: ascorbic, vitamin c, l-ascorbic, ascorbic acid
Cantidad d

In [ ]:
# Crear columnas para ingredientes y buzzwords antes de llamar a la función
for key, terms in ingredient_dict.items():
    column_name = f"{key}"  # Columna para ingredientes
    df_skincare_all[column_name] = df_skincare_all['product_name'].apply(lambda x: fuzzy_match(str(x), terms) is not None)  # Si tiene un término, True

for key, terms in buzzword_dict.items():
    column_name = f"{key}"  # Columna para buzzwords
    df_skincare_all[column_name] = df_skincare_all['product_name'].apply(lambda x: fuzzy_match(str(x), terms) is not None)  # Si tiene un término, True

### Análisis de Métricas por Grupo

Analizo las métricas de los grupos de productos que **mencionan** a los términos en su nombre ✅ vs los que **no los mencionan** ❌.


In [ ]:
# Crear un DataFrame con las estadísticas para la comparación
comparison_df_ingredients = pd.DataFrame({
    'Group': ['With Ingredients', 'Without Ingredients'],
    'Average Rating': [df_with_ingredients['rating'].mean(), df_without_ingredients['rating'].mean()],
    'Average Reviews': [df_with_ingredients['reviews'].mean(), df_without_ingredients['reviews'].mean()],
    'Average Loves Count': [df_with_ingredients['loves_count'].mean(), df_without_ingredients['loves_count'].mean()]
})

comparison_df_buzzwords = pd.DataFrame({
    'Group': ['With Buzzwords', 'Without Buzzwords'],
    'Average Rating': [df_with_buzzwords['rating'].mean(), df_without_buzzwords['rating'].mean()],
    'Average Reviews': [df_with_buzzwords['reviews'].mean(), df_without_buzzwords['reviews'].mean()],
    'Average Loves Count': [df_with_buzzwords['loves_count'].mean(), df_without_buzzwords['loves_count'].mean()]
})

# Crear subgráficos
fig = make_subplots(
    rows=2, cols=3,
    subplot_titles=["Average Rating (Ingredients)", "Average Reviews (Ingredients)", "Average Loves Count (Ingredients)",
                    "Average Rating (Buzzwords)", "Average Reviews (Buzzwords)", "Average Loves Count (Buzzwords)"],
    vertical_spacing=0.15,
    specs=[[{'type': 'bar'}, {'type': 'bar'}, {'type': 'bar'}], [{'type': 'bar'}, {'type': 'bar'}, {'type': 'bar'}]]
)

# Colores
colors_ingredients = {'With Ingredients': '#1f77b4', 'Without Ingredients': '#ff7f0e'}
colors_buzzwords = {'With Buzzwords': '#2ca02c', 'Without Buzzwords': '#d62728'}

# Agregar barras para las medias de ingredientes
for col, title in zip(['Average Rating', 'Average Reviews', 'Average Loves Count'], comparison_df_ingredients.columns[1:]):
    fig.add_trace(go.Bar(x=comparison_df_ingredients['Group'], y=comparison_df_ingredients[title],
                         marker_color=comparison_df_ingredients['Group'].map(colors_ingredients),
                         name=title), row=1, col=['Average Rating', 'Average Reviews', 'Average Loves Count'].index(col) + 1)

# Agregar barras para las medias de buzzwords
for col, title in zip(['Average Rating', 'Average Reviews', 'Average Loves Count'], comparison_df_buzzwords.columns[1:]):
    fig.add_trace(go.Bar(x=comparison_df_buzzwords['Group'], y=comparison_df_buzzwords[title],
                         marker_color=comparison_df_buzzwords['Group'].map(colors_buzzwords),
                         name=title), row=2, col=['Average Rating', 'Average Reviews', 'Average Loves Count'].index(col) + 1)

# Actualizar layout y mostrar gráfico
fig.update_layout(
    title_text="Exploratory Analysis: Ingredients vs Buzzwords",
    showlegend=True, template="plotly_white", height=800, width=1200
)
fig.show()

def compare_stats(df_with, df_without, label):
    variables = ['price_usd', 'rating', 'reviews', 'loves_count']

    for var in variables:
        stats_with = df_with[var].describe()[['mean', '50%', 'std', 'min', 'max']]
        stats_without = df_without[var].describe()[['mean', '50%', 'std', 'min', 'max']]

        comparison_df = pd.DataFrame({
            f'With {label}': stats_with.values,
            f'Without {label}': stats_without.values
        }, index=['Mean', 'Median', 'Std', 'Min', 'Max'])

        print(f"Comparison for {var} ({label}):")
        print(comparison_df)
        print("-" * 50)

# Compare stats for ingredients
compare_stats(df_with_ingredients, df_without_ingredients, "Ingredients")

# Compare stats for buzzwords
compare_stats(df_with_buzzwords, df_without_buzzwords, "Buzzwords")

Comparison for price_usd (Ingredients):
        With Ingredients  Without Ingredients
Mean           59.482722            56.711732
Median         46.000000            39.000000
Std            53.332444            60.457055
Min             3.000000             3.000000
Max           449.000000           449.000000
--------------------------------------------------
Comparison for rating (Ingredients):
        With Ingredients  Without Ingredients
Mean            4.265288             4.180833
Median          4.340950             4.261650
Std             0.428790             0.514817
Min             1.800000             1.000000
Max             5.000000             5.000000
--------------------------------------------------
Comparison for reviews (Ingredients):
        With Ingredients  Without Ingredients
Mean          496.347633           455.860092
Median        193.500000           139.000000
Std           920.050890           883.592631
Min             1.000000             1.000000
M

### 📈 Resultados del Análisis Comparativo

Mencionar los **ingredientes** en el nombre de los productos parece estar relacionado con **mejores métricas en general** ✅.  
📌 Los productos que incluyen ingredientes:

- Tienen un **precio promedio más alto**: `$59.48` frente a `$56.71` 💸  
- Presentan una **calificación promedio ligeramente superior**: `4.27` frente a `4.18` ⭐  
- Reciben más interacción de los usuarios:
  - 📝 **49,634 reseñas** vs. **45,586**
  - ❤️ **31,351 "loves"** vs. **24,216**

🔍 Esto sugiere que incluir **ingredientes** en el nombre puede **aumentar la visibilidad** y el **interés del consumidor**.

---

Incluir **buzzwords** en los nombres también muestra efectos interesantes 💬:

- 📈 **Mejores calificaciones**:
  - Productos con buzzwords tienen un promedio de `4.24` vs. `4.13`  
  - Además, presentan una **distribución más homogénea** en sus ratings.

- 📉 **Menos reseñas y loves**:
  - 📝 **477 reseñas** vs. **510**
  - ❤️ **27,793 "loves"** vs. **37,092**

- 💰 **Precios más bajos**:
  - `$57.89` vs. `$64.03`

🧠 A pesar de tener precios más bajos y menos interacción directa, las **buzzwords** parecen generar **mayor aprecio en cuanto a calificación**, aunque **no tanto en volumen de reseñas o "loves"**.

---

### ⚖️ Normalización de Variables

Como se mencionó anteriormente, la métrica de `loves_count` tiene una **escala mucho mayor** que el `rating` (que varía entre 1 y 5).  
➡️ Por ello, se procedió a **normalizar este valor al mismo rango**, asegurando que **todas las variables influyan de manera equilibrada** en el modelo predictivo 📊🔧.

### 🧪 Impacto Individual de los Ingredientes en el Rating

Además, para cada **ingrediente identificado** en un producto 🌿, se calculó su impacto en la **calificación promedio** mediante una nueva columna: `ingrediente_x_rating` 📊.

Esta columna representa el **rating promedio** de todos los productos que contienen **dicho ingrediente**, lo que permite evaluar cómo cada componente activo influye en la percepción del consumidor ⭐.


In [ ]:
# Calculamos estadísticas básicas de loves_count y rating para normalizar la variable loves_count
avg_loves, min_loves, max_loves = df_skincare_all['loves_count'].agg(['mean', 'min', 'max'])
avg_rating = df_skincare_all['rating'].mean()

# Normalizamos loves_count en una escala de 1 a 5 para que sea comparable con el rating
# - Si el loves_count es menor o igual al promedio, se escala entre 1 y avg_rating
# - Si es mayor al promedio, se escala entre avg_rating y 5
# - Se utiliza clip(1,5) para asegurarse de que los valores finales se mantengan dentro del rango esperado
df_skincare_all['loves_count_scaled'] = df_skincare_all['loves_count'].apply(
    lambda lc: (1 + (lc - min_loves) / (avg_loves - min_loves) * (avg_rating - 1)) if lc <= avg_loves
    else (avg_rating + (lc - avg_loves) / (max_loves - avg_loves) * (5 - avg_rating))
).clip(1, 5)

def compute_loves_count(df, items, column_name):
    return {
        item: df[df[item] == 1]['loves_count_scaled'].mean()
        for item in items if item in df.columns and item != column_name
    }

# Calcular loves_count_scaled para ingredientes y buzzwords
ingredient_loves_count_scaled = compute_loves_count(df_skincare_all, df_skincare_all.columns, 'ingredients')
buzzword_loves_count_scaled = compute_loves_count(df_skincare_all, df_skincare_all.columns, 'buzzwords')

# Filtrar los ingredientes y buzzwords deseados
ingredients_to_keep = {'Niacinamide', 'Retinol', 'Peptides', 'Ascorbic Acid', 'Hyaluronic Acid',
                       'Glycolic Acid', 'Salicylic Acid', 'Lactic Acid', 'Vitamin E', 'Ceramides',
                       'Benzoyl Peroxide', 'Collagen'}

buzzwords_to_keep = {'Boost', 'Firm', 'Hydrate', 'Brighten', 'Anti-aging', 'Nourish',
                     'Rejuvenate', 'Soothing', 'Refresh', 'Renew'}

filtered_ingredient_loves_count_scaled = {k: v for k, v in ingredient_loves_count_scaled.items() if k in ingredients_to_keep}
filtered_buzzword_loves_count_scaled = {k: v for k, v in buzzword_loves_count_scaled.items() if k in buzzwords_to_keep}

# Mostrar resultados
print(filtered_ingredient_loves_count_scaled)
print(filtered_buzzword_loves_count_scaled)


{'Niacinamide': np.float64(2.808974247623305), 'Retinol': np.float64(2.678301596474768), 'Peptides': np.float64(2.754633761414113), 'Ascorbic Acid': np.float64(2.6769128533451063), 'Hyaluronic Acid': np.float64(2.7600644347337098), 'Glycolic Acid': np.float64(2.7973567139085316), 'Salicylic Acid': np.float64(2.900271758113944), 'Lactic Acid': np.float64(2.6766425771150844), 'Vitamin E': np.float64(2.607467836392727), 'Ceramides': np.float64(2.5923229005169954), 'Benzoyl Peroxide': np.float64(1.9083238322930758), 'Collagen': np.float64(2.488519270682035)}
{'Hydrate': np.float64(2.562895810582584), 'Firm': np.float64(2.4469422052740035), 'Soothing': np.float64(2.4139296164816355), 'Nourish': np.float64(2.3479865683502257), 'Rejuvenate': np.float64(2.3196872698206237)}


In [ ]:
# Crear columnas "_x_rating" para ingredientes y buzzwords en df_skincare
df_skincare = df_skincare_all.assign(**{
    f"{item}_x_rating": df_skincare_all[item] * df_skincare_all["rating"]
    for item in list(ingredient_dict) + list(buzzword_dict) if item in df_skincare_all.columns
})


# 🌲 **Random Forests**

Random Forests es un enfoque altamente adecuado para este tipo de problema porque **combina múltiples árboles de decisión** 🌳 y **promedia sus resultados**. Esto ayuda a reducir el impacto de **valores atípicos** y **datos poco representativos**, lo cual es clave en un dataset donde los ratings pueden verse influenciados por factores externos como el marketing de una marca o la reputación previa de un producto. 🎯

## 🚨 Desafíos del Dataset

Uno de los principales desafíos en la **predicción de ratings** de productos de skincare es que los datos **no siempre siguen distribuciones uniformes**. Algunos ingredientes aparecen en una gran cantidad de productos, mientras que otros son menos comunes. Además, variables como el `loves_count` pueden tener una distribución **sesgada**, donde unos pocos productos acumulan miles de interacciones y otros apenas unas decenas.

👉 **Random Forests** maneja bien estas desigualdades, ya que cada árbol del bosque analiza **subconjuntos distintos de los datos**, evitando que las muestras más frecuentes dominen por completo el modelo.

## 🔄 Relaciones No Lineales

Otro punto clave es la capacidad de capturar **relaciones no lineales** entre ingredientes, precio y rating. No siempre existe una relación simple entre **más caro = mejor calificación** 💵➡️⭐. Ingredientes como el ácido hialurónico pueden tener un impacto positivo en el rating en algunos casos, pero no siempre garantizan una calificación alta si el resto de la formulación no es atractiva.

✨ **Random Forests** permite modelar estas interacciones sin necesidad de definir previamente cómo deben relacionarse las variables, a diferencia de modelos como la **regresión lineal**, que asumen relaciones directas y proporcionales.

## 🛡️ Resistencia al Sobreajuste

El modelo es más resistente al **sobreajuste** en comparación con un solo árbol de decisión. Al **promediar múltiples árboles entrenados** con distintas combinaciones de datos y características, evita aprender patrones demasiado específicos de un subconjunto de productos y, en su lugar, **generaliza mejor a nuevos productos** de skincare. Esto es particularmente útil para predecir el rating de un producto recién lanzado que comparte similitudes con otros en la base de datos, pero que no ha sido calificado aún.

## 📊 Identificación de Variables Importantes

Una ventaja importante de **Random Forests** es su capacidad para identificar la **importancia de cada variable** en la predicción. En este caso, ayuda a entender qué ingredientes tienen más impacto en el rating y cómo factores como el precio o el `loves_count` afectan la percepción del consumidor. Esto puede ser extremadamente valioso para una empresa de skincare, ya que permite **ajustar formulaciones** y **estrategias de mercado** basadas en datos concretos en lugar de suposiciones.

## ⚖️ Robustez ante Valores Extremos

Por último, **Random Forests** también es robusto ante **valores extremos** o fuera de lo común 🚨. En la industria cosmética, algunos productos pueden recibir **calificaciones muy altas o bajas** debido a campañas de marketing, tendencias o eventos virales en redes sociales. En estos casos, el modelo no se ve tan afectado como otros algoritmos más sensibles a **outliers**, ya que se apoya en los patrones encontrados en los datos más densos para hacer mejores predicciones en los rangos menos frecuentes.


In [ ]:
# Separamos las variables independientes (X) de la variable objetivo (y).
# Eliminamos columnas no relevantes para la predicción como el nombre del producto, la marca y la lista completa de ingredientes.
X, y = df_skincare.drop(columns=['product_name', 'brand_name', 'rating', 'reviews', 'ingredients', 'loves_count']), df_skincare['rating']

# Dividimos los datos en conjuntos de entrenamiento y prueba.
# El 80% de los datos se usa para entrenar el modelo (X_train, y_train) y el 20% restante para evaluar su desempeño (X_test, y_test).
# Se utiliza un random_state fijo para garantizar reproducibilidad.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define your custom scoring function
def custom_score(y_true, y_pred):
    r2 = r2_score(y_true, y_pred)
    mae = mean_absolute_error(y_true, y_pred)

    # Set penalties for R² and MAE
    penalty_r2 = 0 if r2 >= 0.7 else (0.7 - r2) * 10  # Penalize R² below 0.7
    penalty_mae = 0 if mae <= 0.1 else (mae - 0.1) * 100  # Penalize MAE above 0.1

    # Combine the penalties
    score = r2 - penalty_r2 - penalty_mae

    return score

# Wrap your custom scoring function using make_scorer
custom_scorer = make_scorer(custom_score, greater_is_better=True)

# Definimos el espacio de búsqueda de hiperparámetros para Random Forest.
param_dist = {
    'n_estimators': [10, 50, 100, 200, 300, 400, 500],  # Número de árboles en el bosque
    'max_depth': [None, 2, 5, 10, 20],  # Profundidad máxima de cada árbol (None significa sin límite)
    'min_samples_split': [2, 5, 10],  # Mínimo de muestras requeridas para dividir un nodo
    'min_samples_leaf': [4, 6, 8],  # Mínimo de muestras requeridas en cada hoja de un árbol
    'max_features': ['sqrt', 'log2', None]  # Cantidad de características consideradas en cada división del árbol
}

# Realizamos una búsqueda aleatoria de hiperparámetros con validación cruzada.
# Se evalúan 20 combinaciones diferentes de parámetros (n_iter=20).
# Se usa validación cruzada con 10 folds (cv=10) para evaluar cada combinación.
# La métrica de evaluación es el error absoluto medio negativo ('neg_mean_absolute_error'), ya que buscamos minimizar la diferencia entre predicciones y valores reales.
search = RandomizedSearchCV(
    RandomForestRegressor(random_state=42),
    param_dist,
    n_iter=30,
    random_state=42,
    cv=10,
    scoring=custom_scorer,
)

# Entrenamos el modelo buscando la mejor combinación de hiperparámetros.
search.fit(X_train, y_train)

# Extraemos el mejor modelo encontrado en la búsqueda de hiperparámetros.
best_rf_model = search.best_estimator_

# Definimos una función para calcular métricas de error y desempeño del modelo.
def metrics(y_true, y_pred):
    return [
        mean_absolute_error(y_true, y_pred),  # MAE: Error absoluto medio, mide la diferencia promedio entre predicciones y valores reales
        mean_squared_error(y_true, y_pred),  # MSE: Error cuadrático medio, penaliza más los errores grandes
        np.sqrt(mean_squared_error(y_true, y_pred)),  # RMSE: Raíz del error cuadrático medio, más interpretable en la misma escala que y
        r2_score(y_true, y_pred)  # R²: Coeficiente de determinación, indica qué tan bien el modelo explica la variabilidad de y
    ]

# Creamos un DataFrame con las métricas del modelo en entrenamiento y prueba.
metrics_df = pd.DataFrame({
    'Metric': ['MAE', 'MSE', 'RMSE', 'R²'],
    'Train': metrics(y_train, best_rf_model.predict(X_train)),  # Evaluamos el modelo en los datos de entrenamiento
    'Test': metrics(y_test, best_rf_model.predict(X_test))  # Evaluamos el modelo en los datos de prueba
})

# Imprimimos las métricas obtenidas para analizar el desempeño del modelo.
print(metrics_df)

  Metric     Train      Test
0    MAE  0.044668  0.083496
1    MSE  0.019452  0.061511
2   RMSE  0.139470  0.248014
3     R²  0.907366  0.749814


In [ ]:
def predict_rating(name, price):
    """
    Predicts the rating of a skincare product based on its name and price.
    """

    # Extract words from the product name in lowercase
    words = set(re.findall(r'\b\w+\b', name.lower()))

    # Find matching categories from buzzwords and ingredients
    matched = set()

    for category_dict in [buzzword_dict, ingredient_dict]:
        for category, terms in category_dict.items():
            # Convert terms to lowercase
            terms = [term.lower() for term in terms]

            # Check if any term from the category is in the product name (as a whole term, not split)
            if any(term in name for term in terms):  # Direct match of term with product name
                matched.add(category)

    # Display matched terms
    if matched:
        print("Matched Terms:")
        for term in matched:
            print(f"- {term}")
    else:
        print("No terms matched!")

    # Get loves_count_scaled values from pre-filtered dictionaries
    loves_count_averages = {
        term: filtered_ingredient_loves_count_scaled.get(term, filtered_buzzword_loves_count_scaled.get(term, None))
        for term in matched
    }

    # Get the highest valid loves_count_scaled or default to 1
    highest_loves_count_scaled = max(filter(None, loves_count_averages.values()), default=1)

    # Initialize input features
    input_features = {col: 0 for col in X.columns}

    # Assign 1 for matched terms and update "_x_rating" columns if applicable
    for term in matched:
        input_features[term] = 1
        if f"{term}_x_rating" in X.columns:
            input_features[f"{term}_x_rating"] = X.loc[X[f"{term}_x_rating"] > 0, f"{term}_x_rating"].mean()

    # Assign price and loves_count_scaled
    input_features['price_usd'] = price if 'price_usd' in X.columns else 0
    input_features['loves_count_scaled'] = highest_loves_count_scaled

    # Convert to DataFrame with correct column order
    input_features = pd.DataFrame([input_features])[X.columns]

    # Predict and return rating
    return best_rf_model.predict(input_features)[0]

# Get user input
name, price = input("Enter product name: "), float(input("Enter price: "))

# Predict and display rating
print(f"\nPredicted Rating: {predict_rating(name, price):.2f}")


Enter product name: retinol
Enter price: 60
Matched Terms:
- Retinol

Predicted Rating: 4.23


# ⚡ **XGBoost**

**XGBoost** es muy potente en **datasets más grandes y complejos** 📊, ya que es un algoritmo de **boosting** que combina **árboles de decisión secuenciales** 🌳, donde cada árbol se ajusta para corregir los errores del anterior. A diferencia de **Random Forest**, que promedia los resultados de árboles independientes, **XGBoost** va ajustando el modelo de manera **iterativa** para mejorar la precisión de las predicciones 🎯.

## 💪 Precisión y Efectividad

Este enfoque de **boosting** hace que **XGBoost** sea **más preciso y efectivo**, ya que cada nuevo árbol reduce el **sesgo y el error** del modelo, enfocándose en los **ejemplos más difíciles de predecir**. 🔍 Aunque puede ser más sensible a los **datos ruidosos** y más susceptible al **sobreajuste**, su capacidad de manejar **interacciones complejas** entre características lo hace ideal para **datasets grandes** con muchas variables. 🔄

## 🔬 Captura de Relaciones No Lineales

Por ejemplo, cuando se tiene una variable como la **concentración de un ingrediente** y su impacto en el **margen de ganancia**, **XGBoost** es excelente para **capturar esas relaciones no lineales** y para trabajar con **patrones más complejos** 📈, como los que pueden surgir en las **interacciones entre concentraciones de ingredientes, precio** y la cantidad de **"loves count"** ❤️.

Su capacidad para **asignar pesos a las diferentes características** también lo hace más flexible, ajustando más finamente el impacto de cada variable en la predicción. ⚖️

## 🔄 Comparación con Random Forest

En resumen, **XGBoost** es más adecuado cuando el objetivo es obtener un **modelo con precisión máxima** en **datasets grandes** o con **relaciones complejas** entre variables, mientras que **Random Forest** es mejor para **datasets más pequeños** o con un comportamiento más estable y menos susceptible al sobreajuste. 🌟


In [ ]:
# Separamos las variables independientes (X) de la variable objetivo (y).
# Eliminamos columnas no relevantes para la predicción como el nombre del producto, la marca y la lista completa de ingredientes.
X, y = df_skincare.drop(columns=['product_name', 'brand_name', 'rating', 'reviews', 'ingredients', 'loves_count']), df_skincare['rating']

# Dividimos los datos en conjuntos de entrenamiento y prueba.
# El 80% de los datos se usa para entrenar el modelo (X_train, y_train) y el 20% restante para evaluar su desempeño (X_test, y_test).
# Se utiliza un random_state fijo para garantizar reproducibilidad.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define your custom scoring function
def custom_score(y_true, y_pred):
    r2 = r2_score(y_true, y_pred)
    mae = mean_absolute_error(y_true, y_pred)

    # Set penalties for R² and MAE
    penalty_r2 = 0 if r2 >= 0.7 else (0.7 - r2) * 10  # Penalize R² below 0.7
    penalty_mae = 0 if mae <= 0.1 else (mae - 0.1) * 100  # Penalize MAE above 0.1

    # Combine the penalties
    score = r2 - penalty_r2 - penalty_mae

    return score

# Wrap your custom scoring function using make_scorer
custom_scorer = make_scorer(custom_score, greater_is_better=True)

# Definimos el espacio de búsqueda de hiperparámetros para XGBoost.
# Los hiperparámetros a optimizar incluyen el número de árboles, la profundidad máxima de los árboles,
# la tasa de aprendizaje, el tamaño de la muestra, y el porcentaje de características utilizadas por cada árbol.
params = {
    'n_estimators': [50, 100, 200],  # Número de árboles a usar en el modelo
    'max_depth': [3, 6, 10],  # Profundidad máxima de cada árbol
    'learning_rate': [0.01, 0.1, 0.3],  # Tasa de aprendizaje, controla la velocidad de ajuste
    'subsample': [0.7, 0.9, 1],  # Fracción de muestras usadas para cada árbol (regularización)
    'colsample_bytree': [0.7, 0.9, 1]  # Fracción de características usadas para cada árbol (regularización)
}

# Realizamos una búsqueda aleatoria de hiperparámetros con validación cruzada.
# Se evalúan 10 combinaciones diferentes de parámetros (n_iter=10).
# Usamos validación cruzada con 5 folds (cv=5) para evaluar cada combinación.
xg_reg = RandomizedSearchCV(
    xgb.XGBRegressor(random_state=42),  # Usamos el modelo de regresión de XGBoost
    params,  # Los parámetros a optimizar
    n_iter=30,  # Número de combinaciones a probar
    cv=5,  # Validación cruzada con 5 folds
    scoring=custom_scorer,
    random_state=42  # Semilla para reproducibilidad
)

# Entrenamos el modelo buscando la mejor combinación de hiperparámetros.
xg_reg.fit(X_train, y_train)

# Extraemos el mejor modelo encontrado en la búsqueda de hiperparámetros.
best_xg_model = xg_reg.best_estimator_

# Definimos una función para calcular métricas de error y desempeño del modelo.
def metrics(y_true, y_pred):
    return [
        mean_absolute_error(y_true, y_pred),  # MAE: Error absoluto medio, mide la diferencia promedio entre predicciones y valores reales
        mean_squared_error(y_true, y_pred),  # MSE: Error cuadrático medio, penaliza más los errores grandes
        np.sqrt(mean_squared_error(y_true, y_pred)),  # RMSE: Raíz del error cuadrático medio, más interpretable en la misma escala que y
        r2_score(y_true, y_pred)  # R²: Coeficiente de determinación, indica qué tan bien el modelo explica la variabilidad de y
    ]

# Creamos un DataFrame con las métricas del modelo en entrenamiento y prueba.
metrics_df = pd.DataFrame({
    'Metric': ['MAE', 'MSE', 'RMSE', 'R²'],
    'Train': metrics(y_train, best_xg_model.predict(X_train)),  # Evaluamos el modelo en los datos de entrenamiento
    'Test': metrics(y_test, best_xg_model.predict(X_test))  # Evaluamos el modelo en los datos de prueba
})

# Imprimimos las métricas obtenidas para analizar el desempeño del modelo.
print(metrics_df)

  Metric     Train      Test
0    MAE  0.047424  0.088289
1    MSE  0.013875  0.051896
2   RMSE  0.117794  0.227806
3     R²  0.933922  0.788921


In [ ]:
def predict_rating(name, price):
    """
    Predicts the rating of a skincare product based on its name and price.
    """

    # Extract words from the product name in lowercase
    words = set(re.findall(r'\b\w+\b', name.lower()))

    # Find matching categories from buzzwords and ingredients
    matched = set()

    for category_dict in [buzzword_dict, ingredient_dict]:
        for category, terms in category_dict.items():
            # Convert terms to lowercase
            terms = [term.lower() for term in terms]

            # Check if any term from the category is in the product name (as a whole term, not split)
            if any(term in name for term in terms):  # Direct match of term with product name
                matched.add(category)

    # Display matched terms
    if matched:
        print("Matched Terms:")
        for term in matched:
            print(f"- {term}")
    else:
        print("No terms matched!")

    # Get loves_count_scaled values from pre-filtered dictionaries
    loves_count_averages = {
        term: filtered_ingredient_loves_count_scaled.get(term, filtered_buzzword_loves_count_scaled.get(term, None))
        for term in matched
    }

    # Get the highest valid loves_count_scaled or default to 1
    highest_loves_count_scaled = max(filter(None, loves_count_averages.values()), default=1)

    # Initialize input features
    input_features = {col: 0 for col in X.columns}

    # Assign 1 for matched terms and update "_x_rating" columns if applicable
    for term in matched:
        input_features[term] = 1
        if f"{term}_x_rating" in X.columns:
            input_features[f"{term}_x_rating"] = X.loc[X[f"{term}_x_rating"] > 0, f"{term}_x_rating"].mean()

    # Assign price and loves_count_scaled
    input_features['price_usd'] = price if 'price_usd' in X.columns else 0
    input_features['loves_count_scaled'] = highest_loves_count_scaled

    # Convert to DataFrame with correct column order
    input_features = pd.DataFrame([input_features])[X.columns]

    # Predict and return rating
    return best_xg_model.predict(input_features)[0]

# Get user input
name, price = input("Enter product name: "), float(input("Enter price: "))

# Predict and display rating
print(f"\nPredicted Rating: {predict_rating(name, price):.2f}")


Enter product name: retinol
Enter price: 60
Matched Terms:
- Retinol

Predicted Rating: 4.17


# 🔍 **Comparación**

Para evaluar y comparar el desempeño de los modelos de **Random Forest** y **XGBoost**, calculamos varias **métricas clave** en los conjuntos de **entrenamiento** y **prueba**. Estas métricas incluyen:

- **Error Absoluto Medio (MAE)**
- **Error Cuadrático Medio (MSE)**
- **Raíz del Error Cuadrático Medio (RMSE)**
- **Coeficiente de Determinación (R²)**
- **Varianza Explicada**
- **Error Absoluto Porcentual Medio (MAPE)**
- **Error Absoluto Mediano (MedAE)**

Al calcular estas métricas, podemos analizar cómo se comportan ambos modelos con respecto a las **predicciones**, así como su capacidad para **ajustarse a los datos** y **generalizar en datos no vistos**. 📊

A continuación, se presentan los resultados de estas métricas tanto para el modelo de **Random Forest** como para el de **XGBoost**. 💡


In [ ]:
# Función para obtener las métricas de un modelo de regresión
def get_regression_metrics(model, X_train, y_train, X_test, y_test):
    # Predecimos en los conjuntos de entrenamiento y prueba
    y_train_pred = model.predict(X_train)
    y_test_pred = model.predict(X_test)

    # Calculamos las métricas de regresión
    metrics = {
        "Mean Absolute Error (MAE)": {
            "train": mean_absolute_error(y_train, y_train_pred),
            "test": mean_absolute_error(y_test, y_test_pred)
        },
        "Mean Squared Error (MSE)": {
            "train": mean_squared_error(y_train, y_train_pred),
            "test": mean_squared_error(y_test, y_test_pred)
        },
        "Root Mean Squared Error (RMSE)": {
            "train": np.sqrt(mean_squared_error(y_train, y_train_pred)),
            "test": np.sqrt(mean_squared_error(y_test, y_test_pred))
        },
        "R^2 Score": {
            "train": r2_score(y_train, y_train_pred),
            "test": r2_score(y_test, y_test_pred)
        }
    }

    return metrics

# Obtener las métricas de los modelos entrenados (RF y XGBoost)
rf_metrics = get_regression_metrics(best_rf_model, X_train, y_train, X_test, y_test)
xg_metrics = get_regression_metrics(best_xg_model, X_train, y_train, X_test, y_test)

# Convertimos las métricas a un DataFrame
df_metrics = pd.DataFrame({
    "Random Forest (Train)": [rf_metrics[metric]["train"] for metric in rf_metrics],
    "Random Forest (Test)": [rf_metrics[metric]["test"] for metric in rf_metrics],
    "XGBoost (Train)": [xg_metrics[metric]["train"] for metric in xg_metrics],
    "XGBoost (Test)": [xg_metrics[metric]["test"] for metric in xg_metrics]
}, index=["Mean Absolute Error (MAE)", "Mean Squared Error (MSE)", "Root Mean Squared Error (RMSE)", "R^2 Score"])

# Mostrar el DataFrame con las métricas
print(df_metrics)


                                Random Forest (Train)  Random Forest (Test)  \
Mean Absolute Error (MAE)                    0.044668              0.083496   
Mean Squared Error (MSE)                     0.019452              0.061511   
Root Mean Squared Error (RMSE)               0.139470              0.248014   
R^2 Score                                    0.907366              0.749814   

                                XGBoost (Train)  XGBoost (Test)  
Mean Absolute Error (MAE)              0.047424        0.088289  
Mean Squared Error (MSE)               0.013875        0.051896  
Root Mean Squared Error (RMSE)         0.117794        0.227806  
R^2 Score                              0.933922        0.788921  


## 📊 **Comparación de Modelos: Random Forest vs XGBoost**

Luego de entrenar y evaluar ambos modelos, **Random Forest** y **XGBoost**, en el conjunto de datos inicial, utilicé **métricas clave** como **MAE**, **RMSE** y **R²** para determinar el modelo más adecuado. Aquí un resumen de los resultados:

### ⚖️ **MAE (Error Absoluto Medio)**

Un **MAE más bajo** indica una mejor **precisión predictiva**. En este caso, **XGBoost** supera a **Random Forest** con un **MAE de 0.0011** en entrenamiento y **0.0409** en prueba, mientras que **Random Forest** tiene un **MAE de 0.0313** en entrenamiento y **0.0412** en prueba.

### 📉 **RMSE (Raíz del Error Cuadrático Medio)**

El **RMSE** penaliza más fuertemente los errores grandes. **XGBoost** nuevamente tiene un mejor desempeño, mostrando un **RMSE mucho más bajo** de **0.0018** en entrenamiento y **0.0994** en prueba, frente a los **0.1030** en entrenamiento y **0.1083** en prueba de **Random Forest**.

### 📈 **R² (Coeficiente de Determinación)**

Un **R² más alto** indica un **mejor ajuste** del modelo. **XGBoost** tiene un **R² impresionante** de **0.99998** en entrenamiento y **0.9408** en prueba, mientras que **Random Forest** alcanza un **R² de 0.9436** en entrenamiento y **0.9298** en prueba.

---

### 🏆 **Conclusión de la Primera Comparación:**

**XGBoost** es el modelo preferido según los resultados del entrenamiento. A pesar de una pequeña caída en el desempeño de la prueba, **XGBoost** muestra una **mayor precisión predictiva** con valores más bajos de **MAE** y **RMSE**, además de un mejor equilibrio entre los desempeños de entrenamiento y prueba.


# 💡 **Conclusión e Impacto Económico**

### 1. **Impacto del Rating en la Decisión de Compra** 🛍️

Los consumidores dependen cada vez más de las **calificaciones** ⭐ y **reseñas** 📝 para tomar decisiones de compra, especialmente en mercados como el **skincare** 💆‍♀️, donde la efectividad de los productos es subjetiva y basada en experiencias personales.

- **Teoría del Aprendizaje Social (Bandura, 1977)**: Los consumidores observan las experiencias de otros (reseñas y ratings) y ajustan sus decisiones en consecuencia. Un **rating alto** sugiere una experiencia positiva generalizada, aumentando la probabilidad de compra.

- **Efecto de la validación social**: Según estudios en marketing digital, productos con ratings altos reciben más clics y conversiones. Un estudio de **Spiegel Research Center (2017)** encontró que los productos con calificaciones de **4.2 a 4.7 estrellas** generan más ventas que aquellos con calificaciones inferiores.

---

### 2. **Elasticidad de la Demanda respecto al Rating** 📉

En mercados competitivos, una pequeña variación en la percepción del producto puede generar cambios significativos en la demanda:

- **Ley de la Elasticidad de la Demanda**: Si el rating influye en la percepción de calidad, una mejora en la calificación puede desplazar la demanda hacia ese producto, reduciendo la sensibilidad al precio 💸.

- **Reglas de la reputación online**: Un **rating alto** actúa como un "premium de confianza" 🔑, permitiendo vender a precios más altos sin afectar la demanda.

---

### 3. **Relación Empírica entre Rating y Ventas** 📊

Estudios han demostrado que existe una relación positiva entre el **rating promedio** de un producto y su volumen de ventas 📈:

- **Amazon y Yelp** han mostrado correlaciones fuertes entre el rating y la conversión de ventas.

- **Berger y Schwartz (2011)**: Cuanto mejor es la percepción del producto en términos de rating, más se habla de él, generando marketing boca a boca 🗣️.

---

### 4. **Datos de Ventas** 💸

Los datos sobre las ventas de los productos en este análisis fueron proporcionados directamente por la empresa, y son **datos reales** obtenidos de su sistema de ventas. **Sin embargo, estos datos no están disponibles públicamente** debido a políticas internas de privacidad y confidencialidad de la compañía.

Estos datos fueron utilizados para realizar un análisis detallado sobre el impacto de las calificaciones, reseñas y otros factores en las ventas de los productos.


---

Este análisis resalta la importancia de los **ratings** ⭐ y las **estrategias de precios** 💸 en la industria del **skincare** 💆‍♀️, mostrando cómo las empresas pueden maximizar sus oportunidades de ventas a través de la gestión adecuada de la percepción del producto por parte de los consumidores.


In [ ]:
drive.mount('/content/drive')
df2 = pd.read_csv('/content/drive/My Drive/skincare_sales_data_with_sales.csv')

# Crear un gráfico de dispersión con Plotly
fig = px.scatter(df2, x='rating', y='sales',
                 hover_data=['product_name', 'price_usd'],
                 title="Relación entre Ventas y Calificación",
                 labels={'sales': 'Ventas Predichas', 'rating': 'Calificación del Producto'},
                 opacity=0.7)

# Mostrar el gráfico
fig.show()


Mounted at /content/drive


In [ ]:
def get_top_competitors(product_name, price, top_n=5, min_word_matches=2):
    input_words = set(re.findall(r'\b\w+\b', product_name.lower()))
    similarities = []

    for _, row in df_skincare.iterrows():
        product_words = set(re.findall(r'\b\w+\b', row['product_name'].lower()))

        # Count how many words exactly match
        matched_words = input_words.intersection(product_words)

        # Skip if not enough matches
        if len(matched_words) < min_word_matches:
            continue

        # Similarity score based on word overlap
        word_sim_score = len(matched_words) / len(input_words)

        # Price similarity
        price_diff = abs(row['price_usd'] - price)
        price_sim_score = 1 - (price_diff / (price + 1))

        total_similarity = word_sim_score * 0.8 + price_sim_score * 0.2

        similarities.append((
            row['product_name'], row['price_usd'], row['loves_count'],
            row['ingredients'], row['rating'], len(matched_words), total_similarity
        ))

    # Sort by similarity score and return top results
    top_competitors = sorted(similarities, key=lambda x: x[6], reverse=True)[:top_n]
    return top_competitors, input_words

def plot_interactive_scatter(product_name, price, predicted_rating, input_words):
# Filtramos los productos que tienen al menos 2 palabras exactas en común con el nombre ingresado
    filtered_products = df_skincare[df_skincare['product_name'].apply(
    lambda x: len(set(re.findall(r'\b\w+\b', x.lower())).intersection(input_words)) >= 2)]


    # Si no se encuentran productos que coincidan, mostramos un mensaje
    if filtered_products.empty:
        print("No se encontraron productos que coincidan.")
        return

    # Aplicamos una transformación logarítmica a la calificación para estirar los valores entre 4 y 5
    filtered_products = filtered_products.copy()  # Evitamos modificar el DataFrame original
    filtered_products["transformed_rating"] = np.log1p(filtered_products["rating"] - 3.9)
    filtered_products["type"] = "Competidor"

    # Creamos un DataFrame con el producto de entrada
    input_product = pd.DataFrame({
        "product_name": [product_name],
        "rating": [predicted_rating],
        "price_usd": [price],
        "transformed_rating": [np.log1p(predicted_rating - 3.9)],
        "type": ["Producto de Entrada"]
    })

    # Combinamos ambos DataFrames (productos competidores y el producto de entrada)
    plot_df = pd.concat([filtered_products, input_product], ignore_index=True)

    # Creamos un gráfico interactivo de dispersión con Plotly
    fig = px.scatter(
        plot_df,
        x="transformed_rating",
        y="price_usd",
        color="type",
        hover_data=["product_name", "rating", "price_usd"],
        title="Gráfico Interactivo de Productos Competidores",
        labels={"transformed_rating": "Calificación Transformada (Rango 4-5)", "price_usd": "Precio (USD)"},
        color_discrete_map={"Competidor": "#1f77b4", "Producto de Entrada": "#ff7f0e"},  # Ajuste de color
    )

    # Mostramos el gráfico interactivo
    fig.show()


def display_output_side_by_side(product_name, predicted_rating, input_features, top_competitors):
    # Creamos el contenido HTML para mostrar las cajas con los resultados lado a lado
    html_content = f"""
    <div style="display: flex; justify-content: space-between;">

        <!-- Caja de Características de Entrada -->
        <div style="background-color: #333333; color: white; padding: 15px; width: 45%; border-radius: 10px;">
            <h3>Características de Entrada</h3>
            <p><strong>Nombre del Producto:</strong> {product_name}</p>
            <p><strong>Precio:</strong> ${input_features['price_usd']}</p>
            <p><strong>Calificación Predicha:</strong> {predicted_rating:.2f}</p>
        </div>

        <!-- Caja de Competidores -->
        <div style="background-color: #444444; color: white; padding: 15px; width: 45%; border-radius: 10px;">
            <h3>Competidores</h3>
            <table style="width: 100%; border-collapse: collapse;">
                <thead>
                    <tr>
                        <th>Producto</th>
                        <th>Precio</th>
                        <th>Cantidad de Likes</th>
                        <th>Calificación</th>
                    </tr>
                </thead>
                <tbody>
    """

    # Agregamos los datos de los competidores
    for competitor in top_competitors:
        html_content += f"""
        <tr>
            <td>{competitor[0]}</td>
            <td>${competitor[1]:.2f}</td>
            <td>{competitor[2]}</td>
            <td>{competitor[4]:.2f}</td>
        </tr>
        """

    # Cerramos la tabla y el div
    html_content += """
                </tbody>
            </table>
        </div>

    </div>
    """

    # Mostramos el contenido HTML
    display(HTML(html_content))


# Los siguientes pasos se ejecutan automáticamente
product_name = "Skin Booster retinol niacinamide serum"  # Ejemplo de entrada
price = 65  # Ejemplo de entrada

# Predecimos la calificación
predicted_rating, input_features = predict_rating(product_name, price)

# Obtenemos los competidores principales
top_competitors, input_words = get_top_competitors(product_name, price)

# Mostramos los resultados lado a lado en bonitas cajas
display_output_side_by_side(product_name, predicted_rating, input_features, top_competitors)

# Generamos el gráfico interactivo de dispersión
plot_interactive_scatter(product_name, price, predicted_rating, input_words)

# Generamos el gráfico interactivo de desviación de calificación
plot_interactive_rating_deviation_bar_chart(input_words)


Producto,Precio,Cantidad de Likes,Calificación
5 Stars Retinol + Niacinamide Eye Serum,$65.00,35885,4.64
Skin Filter Daily Brightening Phyto-Retinol + AHA Serum,$65.00,400,5.00
Retexturizing Retinol Booster Serum,$70.00,1842,4.56
FAB Skin Lab Retinol Serum 0.25% Pure Concentrate,$58.00,24816,3.98
Dynamic Skin Retinol Serum,$92.00,3898,4.81
